In [1]:
def analytic_xcor_XY(t,tx,ty,gy):
    values=np.zeros((len(t),1)).ravel()
    tg0=t[t>0]
    R_AA=np.sqrt(np.pi/2)*(1/(2*tx))
    R_BB = (gy**2*ty**2*np.sqrt(np.pi/2))*((ty-tx)/(2*(ty**2-tx**2)))# maybe no 2x?0.0
    NAB=1/np.sqrt(R_AA*R_BB)
    values[t>0]=.5*gy*ty*np.sqrt(np.pi/2)*np.exp(-tg0/tx)/(tx+ty)
    tl0 = t[t <= 0]
    #normalization factors
    # values[t <= 0]=gy*ty*np.sqrt(np.pi/2)*(2*ty*np.exp(tl0/tx)//(ty**2-tx**2))
    values[t <= 0] =.5*gy*ty*np.sqrt(np.pi/2)*(((2*ty*np.exp(tl0/ty))/(ty**2-tx**2))-(np.exp(tl0/tx))/(ty-tx))
    return values*NAB

In [2]:
import numpy as np

In [3]:
def Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z):
    dt = float(t_end - t_int) / N

    ### Initialize Matrix for all possible cells at all time points for all dimension of data (lets say we're not going to let it get more than 100)
    Intrinsic_Noise = np.empty((N, 3))
    Intrinsic_Noise[:,:]= np.NAN

    Gene = np.empty((N, 3))
    Gene[:,:]= np.NAN


    g_y=1
    g_z=1

    lambda_x=(1.0 - np.exp(-dt / tau_x))

    ### initializing values

    cellular_indexes = np.array(range(100))
    # for index in cellular_indexes:
    Gene[0, 0] = 0
    Gene[0, 1] = 0
    Gene[0, 2] = 0

    for t in np.arange(N-1)+1:
        Gene[t,0] = Gene[t - 1,0] + dt * (-Gene[t - 1,0]/tau_x)+np.sqrt(2/tau_x)*np.random.normal(scale=np.sqrt(dt),loc=0.0)
        Gene[t,1] = Gene[t - 1,1] + dt * (g_y*Gene[t - 1,0]-Gene[t - 1,1]/tau_y)+np.sqrt(2/tau_y)*np.random.normal(scale=np.sqrt(dt),loc=0.0)
        Gene[t, 2] = Gene[t - 1,2] + dt * (g_z*Gene[t - 1,0]-Gene[t - 1,2]/tau_z)+np.sqrt(2/tau_z)*np.random.normal(loc=0.0,scale=np.sqrt(dt))
        # Gene[t,0] = Gene[t - 1,0] + dt * (-Gene[t - 1,0]/tau_x+np.sqrt(2/tau_x)*np.random.normal(loc=0.0))
        # Gene[t,1] = Gene[t - 1,1] + dt * (g_y*Gene[t - 1,0]-Gene[t - 1,1]/tau_y+np.sqrt(2/tau_y)*np.random.normal(loc=0.0))
        # Gene[t, 2] = Gene[t - 1,2] + dt * (g_z*Gene[t - 1,0]-Gene[t - 1,2]/tau_z+np.sqrt(2/tau_z)*np.random.normal(loc=0.0))
    return Gene



In [5]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
# import dynamical_driver as dd




t_end = 200000

t_int = 0



N = 400000
dt = float(t_end - t_int) / N
time_vect= np.arange(t_int, t_end, dt)

tau_x=5.00000002/np.log(2)
tau_y=30/np.log(2)
tau_z=30.00001/np.log(2)

Gene = Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z)


# cc_storage1=np.load('X_Y2.npy')
cc_storage1 = np.zeros((len(Gene[:, 0]) * 2, 1))
### calculate x cor for activator and the downstream
cc_storage1[len(Gene[:, 0]):, 0] = sm.tsa.stattools.ccf(Gene[:, 0], Gene[:, 1])
cc_storage1[0:len(Gene[:, 0]), 0] = sm.tsa.stattools.ccf(Gene[:, 1], Gene[:, 0])[::-1]


In [35]:
# from scipy import interp1d
from scipy.interpolate import interp1d
fig, ax1=plt.subplots()
x_vect=np.linspace(-t_end,t_end,2*len(time_vect))
index_vals=np.linspace(-500,500,101)
f = interp1d(x_vect.ravel(), cc_storage1.ravel())

ax1.scatter(index_vals,f(index_vals),alpha=.25,label='X*Y',color='blue')
ax1.plot(x_vect,analytic_xcor_XY(x_vect,tau_x,tau_y,1.0),linewidth=2,color='red')
ax1.set_xlim([-500,500])
ax1.set_ylim([-.2,1.2])

(-0.2, 1.2)

In [36]:
fig.savefig('dots_lines_debug.png')

In [37]:
xcor_df=np.zeros((100,51))

In [40]:
simulation_range=np.linspace(500,100000,100)
index_vals=np.linspace(-500,500,51)
for z,value in enumerate(simulation_range):

    t_end = int(value)
    t_int = 0
    N = int(value*2)
    dt = float(t_end - t_int) / N
    time_vect= np.arange(t_int, t_end, dt)
    Gene = Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z)
    cc_storage1 = np.zeros((len(Gene[:, 0]) * 2, 1))
    cc_storage1[len(Gene[:, 0]):, 0] = sm.tsa.stattools.ccf(Gene[:, 0], Gene[:, 1])
    cc_storage1[0:len(Gene[:, 0]), 0] = sm.tsa.stattools.ccf(Gene[:, 1], Gene[:, 0])[::-1]
    x_vect=np.linspace(-t_end,t_end,2*len(time_vect))
    f = interp1d(x_vect.ravel(), cc_storage1.ravel())
    xcor_df[z,:]=f(index_vals)

In [77]:
from IPython.display import HTML
from matplotlib import animation, rc
fig2,(ax2,ax3) = plt.subplots(1,2,figsize=(10,4))

# ax2.scatter(index_vals,xcor_df[0,:])
ax2.plot(x_vect,analytic_xcor_XY(x_vect,tau_x,tau_y,0.1),linewidth=3,color='red',label='Analytic Solution')
ax2.set_xlim([-500,500])
ax2.set_ylim([-.2,1.2])
ax3.set_xlim([-10,100])
ax3.set_ylim([0.00005,.1])

ax3.set_yscale('log')
ax3.set_ylabel('Error')
ax2.set_xlabel('Tau')
ax2.set_ylabel('Correlation')
dots = ax2.plot([],[],color='blue',linestyle='none',marker='o',label='Numerical Calculation')
line=ax3.plot([],[],color='blue',label='Accuracy',linewidth=3)

timetext=ax2.text(-200,1,'')
index_vals=np.linspace(-500,500,51)
simulation_rank=np.arange(0,100,100)
true_values=analytic_xcor_XY(index_vals,tau_x,tau_y,0.1)

def init():
#     data=np.hstack((index_vals[:],xcor_df[0,:]))
    dots[0].set_data(index_vals[:],xcor_df[0,:])
    line[0].set_data(index_vals[:],xcor_df[0,:])

mean_square_vals=[]
def animate(i):
#     data=np.hstack((index_vals[:],xcor_df[i,:]))
    dots[0].set_data(index_vals[:],xcor_df[i,:])
    timetext.set_text('Length of Simulation:\n'+str(int(simulation_range[i])))
    mean_square_calc=np.sum((true_values-xcor_df[i,:])**2)/len(true_values)
    mean_square_vals.append(mean_square_calc)
    line[0].set_data(np.arange(0,i+1),mean_square_vals)
    ### calculating the accuracy of the simulation
    
    return dots,line,timetext

    

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig2, animate,init_func=init,
                               frames=100, interval=300, blit=False)
HTML(anim.to_html5_video())

/Users/nicholasrossi/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [73]:
true_values



array([  8.71958124e-06,   1.38414724e-05,   2.19719679e-05,
         3.48783250e-05,   5.53658897e-05,   8.78878716e-05,
         1.39513300e-04,   2.21463559e-04,   3.51551486e-04,
         5.58053199e-04,   8.85854236e-04,   1.40620595e-03,
         2.23221280e-03,   3.54341694e-03,   5.62482378e-03,
         8.92885119e-03,   1.41736678e-02,   2.24992950e-02,
         3.57154028e-02,   5.66946396e-02,   8.99966759e-02,
         1.42853545e-01,   2.26649497e-01,   3.57921729e-01,
         5.38374584e-01,   3.77964474e-01,   2.36227799e-02,
         1.47642376e-03,   9.22764859e-05,   5.76728043e-06,
         3.60455031e-07,   2.25284397e-08,   1.40802750e-09,
         8.80017195e-11,   5.50010753e-12,   3.43756724e-13,
         2.14847955e-14,   1.34279973e-15,   8.39249843e-17,
         5.24531158e-18,   3.27831977e-19,   2.04894988e-20,
         1.28059369e-21,   8.00371065e-23,   5.00231921e-24,
         3.12644954e-25,   1.95403099e-26,   1.22126938e-27,
         7.63293371e-29,

In [44]:
len(true_values)


51

In [45]:
1:4
    

SyntaxError: invalid syntax (<ipython-input-45-ef6c64b83adf>, line 1)

In [46]:
[1:4]

SyntaxError: invalid syntax (<ipython-input-46-d1fc8db7b247>, line 1)

In [52]:
np.arange(1,2)

array([1])

In [80]:
xcor_df2=np.zeros((100,51))
for z,value in enumerate(simulation_range):

    t_end = int(value)
    t_int = 0
    N = int(value*2)
    dt = float(t_end - t_int) / N
    time_vect= np.arange(t_int, t_end, dt)
    tau_x=60.000000002/np.log(2)
    Gene = Simplified_simulation(t_int,t_end,N,tau_x,tau_y,tau_z)
    cc_storage1 = np.zeros((len(Gene[:, 0]) * 2, 1))
    cc_storage1[len(Gene[:, 0]):, 0] = sm.tsa.stattools.ccf(Gene[:, 0], Gene[:, 1])
    cc_storage1[0:len(Gene[:, 0]), 0] = sm.tsa.stattools.ccf(Gene[:, 1], Gene[:, 0])[::-1]
    x_vect=np.linspace(-t_end,t_end,2*len(time_vect))
    f = interp1d(x_vect.ravel(), cc_storage1.ravel())
    xcor_df2[z,:]=f(index_vals)

In [82]:
fig3,(ax4,ax5) = plt.subplots(1,2,figsize=(10,4))

# ax2.scatter(index_vals,xcor_df[0,:])
ax4.plot(x_vect,analytic_xcor_XY(x_vect,tau_x,tau_y,0.1),linewidth=3,color='red',label='Analytic Solution')
ax4.set_xlim([-500,500])
ax4.set_ylim([-.2,1.2])
ax5.set_xlim([-10,100])
ax5.set_ylim([0.00005,1])

ax5.set_yscale('log')
ax5.set_ylabel('Error')
ax4.set_xlabel('Tau')
ax4.set_ylabel('Correlation')
dots = ax4.plot([],[],color='blue',linestyle='none',marker='o',label='Numerical Calculation')
line=ax5.plot([],[],color='blue',label='Accuracy',linewidth=3)

timetext=ax4.text(-200,1,'')
index_vals=np.linspace(-500,500,51)
simulation_rank=np.arange(0,100,100)
true_values=analytic_xcor_XY(index_vals,tau_x,tau_y,0.1)

def init2():
#     data=np.hstack((index_vals[:],xcor_df[0,:]))
    dots[0].set_data(index_vals[:],xcor_df[0,:])
    line[0].set_data(index_vals[:],xcor_df[0,:])

mean_square_vals=[]
def animate2(i):
#     data=np.hstack((index_vals[:],xcor_df[i,:]))
    dots[0].set_data(index_vals[:],xcor_df2[i,:])
    timetext.set_text('Length of Simulation:\n'+str(int(simulation_range[i])))
    mean_square_calc=np.sum((true_values-xcor_df2[i,:])**2)/len(true_values)
    mean_square_vals.append(mean_square_calc)
    line[0].set_data(np.arange(0,i+1),mean_square_vals)
    ### calculating the accuracy of the simulation
    
    return dots,line,timetext

    

# call the animator. blit=True means only re-draw the parts that have changed.
anim2 = animation.FuncAnimation(fig3, animate2,init_func=init2,
                               frames=100, interval=300, blit=False)
HTML(anim2.to_html5_video())

/Users/nicholasrossi/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
